### Problem statement, scope
#### 1. Objective
<!-- trying to frame problem statement, define scope -->
Design and demonstrate an **agentic workflow** capable of answering user queries related to Singapore public transport. Primary focus will be on **real-time bus arrival information**.
Agent will dynamically decide when, how to fetch data from APIs and generate responses via LLM.

#### 2. Agent responsibilities
- interpret user queries (in natural language)
- identify user intent, relevant entities
- dynamically invoke LTA APIs
- ?? incorporate constraints depending on context ?? (eg - time of the day, weather conditions?, holidays?)
- generate llm response
- use langgraph to showcase clear, modular workflow


### Tools, API Selection

#### 1. External data source - LTA DataMall APIs
agent will rely on LTA DataMall as the main source for real time Singapore public transport data

#### 2. Selected APIs
- **Bus arrival API*** : provides real time estimated arrival time for buses at given stop
- Most common user intent : 'when is the next bus arriving?'

- **Bus stops API** : provides metadata about bus stops

#### 3. Tools to support contextual reasoning
- **Time context tool** : extract current time, categorize it
- **Weather context tool** : simulate (randomize) / externally fetch weather conditions
- **Holiday/event rules**

### Agentic Workflow - LangGraph
- Decision driven workflow, stateful instead of a single LLM call.

- **Agent state components**:
  - **user_query** : user input in natural language
  - **intent** : intent inferred from user query (eg - bus arrival, service delayed etc.)
  - **entities** : bus stop (name or code?), time references
  - **context** : time of day, weather, holidays or special events
  - **planned_tools** : tools, APIs agent decided to invoke
  - **tool_results** : output returned by tools
  - **final_response** : final output generated for user

  - **Workflow Nodes** - agent workflow nodes orchestrated by LangGraph
    - **interpret input node** - analyze raw query, infer intent, extract relevant entities
    - **context enrichment node** - augment state with context not stated explicitly by user
    - **planning node** - decide which tools/api to invoke, order of tool execution
    - **tool execution node** - invoke selected tools/apis
    - **response creation node** - combine tool results+context, generate response via LLM 

### LangGraph Skeleton Implementation

In [2]:
from typing import TypedDict, List, Dict, Any
from langgraph.graph import StateGraph, END

# agent state
# using typed dict to make value types consistent
class AgentState(TypedDict):
    user_query: str
    intent: str
    entities: Dict[str, Any]
    context: Dict[str, Any]
    planned_tools: List[str]
    tool_results: Dict[str, Any]
    final_response: str

# node to interpret user input
def interpret_input(state: AgentState) -> AgentState:
    # interpret raw user query to extract intent, entities
    # use placeholder for now
    return {
        **state,
        "intent": "bus_arrival_query",
        "entities": {}
    }

# node to enrich context - time ofday, weather etc.
def enrich_context(state: AgentState) -> AgentState:
    context = {
        "time_of_day": "peak",
        "weather": "clear",
        "is_holiday": False
    }

    return {
        **state,
        "context": context
    }

# planning node - decide which tools, apis to be invoked
# decision made based on interpreted intent, entities
def plan_tools(state: AgentState) -> AgentState:
    planned_tools = []

    if state["intent"] == "bus_arrival_query":
        planned_tools.append("bus_arrival_api")

    return {
        **state,
        "planned_tools": planned_tools
    }

# tool execution node
def execute_tools(state: AgentState) -> AgentState:
    tool_results = {}

    for tool in state["planned_tools"]:
        tool_results[tool] = "placeholder_result"

    return {
        **state,
        "tool_results": tool_results
    }

# final response node
def synthesize_response(state: AgentState) -> AgentState:
    response = "dummy response"

    return {
        **state,
        "final_response": response
    }


In [3]:
# Graph construction
graph = StateGraph(AgentState)

graph.add_node("interpret_input", interpret_input)
graph.add_node("enrich_context", enrich_context)
graph.add_node("plan_tools", plan_tools)
graph.add_node("execute_tools", execute_tools)
graph.add_node("synthesize_response", synthesize_response)

graph.set_entry_point("interpret_input")

graph.add_edge("interpret_input", "enrich_context")
graph.add_edge("enrich_context", "plan_tools")
graph.add_edge("plan_tools", "execute_tools")
graph.add_edge("execute_tools", "synthesize_response")
graph.add_edge("synthesize_response", END)

agent = graph.compile()


In [6]:
initial_state: AgentState = {
    "user_query": "When is the next bus arriving at Orchard Road?",
    "intent": "",
    "entities": {},
    "context": {},
    "planned_tools": [],
    "tool_results": {},
    "final_response": ""
}

result = agent.invoke(initial_state)
result["final_response"]


'dummy response'

### Integrate Tools using LTA DataMall datasets

use JSON datasets provided by LTA DataMall

In [8]:
# LOAD DATASET

import json
from pathlib import Path
from typing import Optional

# DATA_DIR = Path(__file__).parent.resolve() / "data"
DATA_DIR_PATH = Path("data")

# load json dataset from disk
def load_json_dataset(filename: str) -> dict:
    path = DATA_DIR_PATH / filename
    with open(path, "r") as f:
        return json.load(f)


In [ ]:
# BUS STOP SEARCH TOOL
def search_bus_stop(query: str) -> Optional[Dict[str, Any]]:
    data = load_json_dataset("bus_stops.json")
    stops = data.get("value", [])

    query_lower = query.lower()

    for stop in stops:
        if (
            query_lower == stop.get("BusStopCode")
            or query_lower in stop.get("Description", "").lower()
            or query_lower in stop.get("RoadName", "").lower()
        ):
            return {
                "bus_stop_code": stop["BusStopCode"],
                "description": stop["Description"],
                "road_name": stop["RoadName"]
            }

    return None
